In [1]:
#Importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load Data

In [2]:
#Loading data into dataframes
character_df = pd.read_csv("fixed_characters.csv") #Do we need encoding ="latin-1"?
lines_df = pd.read_csv("fixed_lines.csv", encoding ="latin-1") 


In [3]:
lines_df

,Unnamed: 0,lineId,chId,mId,chName,dialogue
0,0,L1045,u0,m0,BIANCA,They do not
1,1,L1044,u2,m0,CAMERON,They do to
2,2,L985,u0,m0,BIANCA,I hope so
3,3,L984,u2,m0,CAMERON,She okay
4,4,L925,u0,m0,BIANCA,Let's go
...,...,...,...,...,...,...
292938,293197,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...
292939,293198,L666370,u9034,m616,VEREKER,I'm to take the Sikali with the main column to...
292940,293199,L666369,u9030,m616,DURNFORD,Your orders Mr Vereker
292941,293200,L666257,u9030,m616,DURNFORD,Good ones yes Mr Vereker Gentlemen who can rid...


In [4]:
character_df.head()

,id,name,movie_id,title,gender,credit_position
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,M,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,F,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [6]:
character_df.shape

(9034, 6)

# Normalize Data


In [7]:
# Change all text in name to lowercase
character_df.name = character_df.name.str.lower()
character_df

,id,name,movie_id,title,gender,credit_position
0,u0,bianca,m0,10 things i hate about you,f,4
1,u1,bruce,m0,10 things i hate about you,M,?
2,u2,cameron,m0,10 things i hate about you,m,3
3,u3,chastity,m0,10 things i hate about you,F,?
4,u4,joey,m0,10 things i hate about you,m,6
...,...,...,...,...,...,...
9029,u9030,durnford,m616,zulu dawn,?,?
9030,u9031,melvill,m616,zulu dawn,?,?
9031,u9032,norris-newman,m616,zulu dawn,M,?
9032,u9033,stuart smith,m616,zulu dawn,?,?


In [8]:
# Fix the tabs and spacing in rows with errors
def reset_tabs(row):
  character_id = row['id']
  name = row['name']
  movie_id = row['movie_id']
  title = row['title']
  gender = row['gender']
  credit_position = row['credit_position']
  if '\t' in character_id:
    lst = character_id.split('\t')
    character_id = lst[0]
    name = lst[1]
    movie_id = lst[2]
    title = lst[3]
    gender = lst[4]
    credit_position = lst[5]
    print(character_id)
  series = pd.Series([character_id, name, movie_id, title, gender, credit_position])
  return series

In [9]:
character_df = character_df.apply(reset_tabs, axis=1)

u1602
u1866
u4660
u4664
u5011
u6736
u6737
u6738
u6739
u6740
u6741
u6742
u6743
u7230
u7845
u7914
u7917


In [10]:
character_df
character_df[character_df[1].isnull()]
character_df.dropna()

,0,1,2,3,4,5
0,u0,bianca,m0,10 things i hate about you,f,4
1,u1,bruce,m0,10 things i hate about you,M,?
2,u2,cameron,m0,10 things i hate about you,m,3
3,u3,chastity,m0,10 things i hate about you,F,?
4,u4,joey,m0,10 things i hate about you,m,6
...,...,...,...,...,...,...
9029,u9030,durnford,m616,zulu dawn,?,?
9030,u9031,melvill,m616,zulu dawn,?,?
9031,u9032,norris-newman,m616,zulu dawn,M,?
9032,u9033,stuart smith,m616,zulu dawn,?,?


In [11]:
character_df = character_df.rename(columns={
    0: 'id',
    1: 'name',
    2: 'movie_id',
    3: 'title',
    4: 'gender',
    5: 'credit_position'
})

character_df

,id,name,movie_id,title,gender,credit_position
0,u0,bianca,m0,10 things i hate about you,f,4
1,u1,bruce,m0,10 things i hate about you,M,?
2,u2,cameron,m0,10 things i hate about you,m,3
3,u3,chastity,m0,10 things i hate about you,F,?
4,u4,joey,m0,10 things i hate about you,m,6
...,...,...,...,...,...,...
9029,u9030,durnford,m616,zulu dawn,?,?
9030,u9031,melvill,m616,zulu dawn,?,?
9031,u9032,norris-newman,m616,zulu dawn,M,?
9032,u9033,stuart smith,m616,zulu dawn,?,?


In [12]:
character_df['gender'].value_counts()

M    2456
?    2418
m    1899
F    1340
f     921
Name: gender, dtype: int64

In [13]:
character_df.dtypes

id                 object
name               object
movie_id           object
title              object
gender             object
credit_position    object
dtype: object

In [14]:
character_df['gender'][character_df.gender == '?'] = "N"

In [15]:
character_df['gender'].value_counts()

M    2456
N    2418
m    1899
F    1340
f     921
Name: gender, dtype: int64

In [16]:
character_df['gender'] = character_df['gender'].str.upper()

In [17]:
character_df = character_df.rename(columns={
   'id': 'chid',
})
character_df

,chid,name,movie_id,title,gender,credit_position
0,u0,bianca,m0,10 things i hate about you,F,4
1,u1,bruce,m0,10 things i hate about you,M,?
2,u2,cameron,m0,10 things i hate about you,M,3
3,u3,chastity,m0,10 things i hate about you,F,?
4,u4,joey,m0,10 things i hate about you,M,6
...,...,...,...,...,...,...
9029,u9030,durnford,m616,zulu dawn,N,?
9030,u9031,melvill,m616,zulu dawn,N,?
9031,u9032,norris-newman,m616,zulu dawn,M,?
9032,u9033,stuart smith,m616,zulu dawn,N,?


In [18]:
gender_count = character_df['gender'].value_counts()
gender_count

M    4355
N    2418
F    2261
Name: gender, dtype: int64

In [19]:
#Cleaning all of the hyphens out of the dialogue
newlines_df = lines_df
newlines_df['dialogue'] = newlines_df['dialogue'].replace('-', ' ', regex=True).astype(str)
newlines_df

,Unnamed: 0,lineId,chId,mId,chName,dialogue
0,0,L1045,u0,m0,BIANCA,They do not
1,1,L1044,u2,m0,CAMERON,They do to
2,2,L985,u0,m0,BIANCA,I hope so
3,3,L984,u2,m0,CAMERON,She okay
4,4,L925,u0,m0,BIANCA,Let's go
...,...,...,...,...,...,...
292938,293197,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...
292939,293198,L666370,u9034,m616,VEREKER,I'm to take the Sikali with the main column to...
292940,293199,L666369,u9030,m616,DURNFORD,Your orders Mr Vereker
292941,293200,L666257,u9030,m616,DURNFORD,Good ones yes Mr Vereker Gentlemen who can rid...


In [20]:
#Cleaning all of the double hyphens out of the dialogue
newlines_df['dialogue'] = newlines_df['dialogue'].replace('--', ' ', regex=True).astype(str)
newlines_df

,Unnamed: 0,lineId,chId,mId,chName,dialogue
0,0,L1045,u0,m0,BIANCA,They do not
1,1,L1044,u2,m0,CAMERON,They do to
2,2,L985,u0,m0,BIANCA,I hope so
3,3,L984,u2,m0,CAMERON,She okay
4,4,L925,u0,m0,BIANCA,Let's go
...,...,...,...,...,...,...
292938,293197,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...
292939,293198,L666370,u9034,m616,VEREKER,I'm to take the Sikali with the main column to...
292940,293199,L666369,u9030,m616,DURNFORD,Your orders Mr Vereker
292941,293200,L666257,u9030,m616,DURNFORD,Good ones yes Mr Vereker Gentlemen who can rid...


In [21]:
# testing to see if they were replaced
newlines_df['dialogue'].str.findall('--').value_counts

<bound method IndexOpsMixin.value_counts of 0         []
1         []
2         []
3         []
4         []
          ..
292938    []
292939    []
292940    []
292941    []
292942    []
Name: dialogue, Length: 292943, dtype: object>

In [22]:
newlines_df = newlines_df[['lineId', 'chId', 'mId', 'chName', 'dialogue']]
newlines_df

,lineId,chId,mId,chName,dialogue
0,L1045,u0,m0,BIANCA,They do not
1,L1044,u2,m0,CAMERON,They do to
2,L985,u0,m0,BIANCA,I hope so
3,L984,u2,m0,CAMERON,She okay
4,L925,u0,m0,BIANCA,Let's go
...,...,...,...,...,...
292938,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...
292939,L666370,u9034,m616,VEREKER,I'm to take the Sikali with the main column to...
292940,L666369,u9030,m616,DURNFORD,Your orders Mr Vereker
292941,L666257,u9030,m616,DURNFORD,Good ones yes Mr Vereker Gentlemen who can rid...


In [23]:
newlines_df = newlines_df.rename(columns={
   'chId': 'chid',
})
newlines_df

,lineId,chid,mId,chName,dialogue
0,L1045,u0,m0,BIANCA,They do not
1,L1044,u2,m0,CAMERON,They do to
2,L985,u0,m0,BIANCA,I hope so
3,L984,u2,m0,CAMERON,She okay
4,L925,u0,m0,BIANCA,Let's go
...,...,...,...,...,...
292938,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...
292939,L666370,u9034,m616,VEREKER,I'm to take the Sikali with the main column to...
292940,L666369,u9030,m616,DURNFORD,Your orders Mr Vereker
292941,L666257,u9030,m616,DURNFORD,Good ones yes Mr Vereker Gentlemen who can rid...


# Merge Datasets

In [24]:
#Merge the datasets so that every line has a gender associated with it
characterline_df = newlines_df.merge(character_df, on='chid', how='left')
characterline_df.head()

,lineId,chid,mId,chName,dialogue,name,movie_id,title,gender,credit_position
0,L1045,u0,m0,BIANCA,They do not,bianca,m0,10 things i hate about you,F,4
1,L1044,u2,m0,CAMERON,They do to,cameron,m0,10 things i hate about you,M,3
2,L985,u0,m0,BIANCA,I hope so,bianca,m0,10 things i hate about you,F,4
3,L984,u2,m0,CAMERON,She okay,cameron,m0,10 things i hate about you,M,3
4,L925,u0,m0,BIANCA,Let's go,bianca,m0,10 things i hate about you,F,4


In [25]:
#Normalize the dialogue in the new refined dataset
slim_df = characterline_df[['gender', 'dialogue']]
slim_df['dialogue'] = slim_df['dialogue'].apply(lambda x: x.lower())
slim_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,gender,dialogue
0,F,they do not
1,M,they do to
2,F,i hope so
3,M,she okay
4,F,let's go
...,...,...
292938,N,lord chelmsford seems to want me to stay back ...
292939,N,i'm to take the sikali with the main column to...
292940,N,your orders mr vereker
292941,N,good ones yes mr vereker gentlemen who can rid...


In [26]:
slim_df.gender.fillna('N', inplace =True)


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [27]:
slim_df.gender.isna().sum()

0

# Encode the gender labels

In [28]:
# slim_df.gender = [ 1 if each == 'f' elif each == 'M'{ slim_df.gender = 2} else 0 for each data.gender]
slim_df['gender'] = slim_df['gender'].replace('M', 0, regex=True)
slim_df['gender'] = slim_df['gender'].replace('F', 1, regex=True)
slim_df['gender'] = slim_df['gender'].replace('N', 2, regex=True)
slim_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,gender,dialogue
0,1,they do not
1,0,they do to
2,1,i hope so
3,0,she okay
4,1,let's go
...,...,...
292938,2,lord chelmsford seems to want me to stay back ...
292939,2,i'm to take the sikali with the main column to...
292940,2,your orders mr vereker
292941,2,good ones yes mr vereker gentlemen who can rid...


In [29]:
# slim_df['gender'] = slim_df['gender'].astype(int)
slim_df.dtypes

gender       int64
dialogue    object
dtype: object

In [30]:
#Looking at the counts for each gender
slim_df.gender.value_counts().values

array([187727,  82221,  22995])

In [31]:
#Calculating the male/female counts - men get more than double the lines than women
# make a chart
slim_df.gender.value_counts().values[0]/slim_df.gender.value_counts().values[1]


2.28320015567799

In [33]:
characterline_df['name'].nunique()

5231

# Remove Stopwords

In [35]:
# Remove Stopwords
import nltk
!pip install --upgrade nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [36]:
type(stopwords.words('english'))

list

### Create new stopwords set to include character names

In [37]:
sw = stopwords.words('english')
names = character_df['name']
names = names.to_list()

sw.extend(names)
print(sw)



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [38]:
# what type is sw?
type(sw)

list

### Sample Data to try removing stopwords and lemmatization

In [76]:
#Create sample data from a line of dialogue
sample_text = slim_df.dialogue[292940]
sample_text

'your orders mr vereker'

In [77]:
#Remove the stopwords from the sample data

#download the nltk words
nltk.download('punkt')
#text  = text .split()
sample_text  = nltk.word_tokenize(sample_text )
#if we use word_tokenize instead of split it will be better
#split() = shouldn't => shouldn't
#word_tokenize() = shouldn't => shouldn't and n't separate as two word
sample_text  = [word for word in sample_text  if not word in set(sw)]
sample_text 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['orders', 'mr']

In [78]:
type(sample_text)

list

In [79]:
#Stemming words or lemmatazation
import nltk as nlp
from nltk.corpus import wordnet

# lemma = nlp.WordNetLemmatizer
# sample_text = [lemma.lemmatize(word) for word in sample_text]
# sample_text

# NOT WORKING - 
# TypeError: lemmatize() missing 1 required positional argument: 'word'

In [65]:
!pip install --upgrade nltk

     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434673 sha256=474d1c6cd02ea004042d7bd1c0204028abff28afaf3ae4768b06af483cd791b0
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [80]:
#Stemming words
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

sample_text = [porter_stemmer.stem(word) for word in sample_text]
sample_text

['order', 'mr']

In [82]:
#Turn words into one string
sample_text = " ".join(sample_text)
sample_text

'order mr'

# Remove stopwords and stem every line of dialogue

In [ ]:

sample_text  = nltk.word_tokenize(sample_text )
#if we use word_tokenize instead of split it will be better
#split() = shouldn't => shouldn't
#word_tokenize() = shouldn't => shouldn't and n't separate as two word
sample_text  = [word for word in sample_text  if not word in set(sw)]
 

porter_stemmer = PorterStemmer()

sample_text = [porter_stemmer.stem(word) for word in sample_text]
sample_text

In [88]:
len(slim_df.dialogue)

292943

In [83]:
import time
dialogue_list = []

count = 0
start_time =  time.time()
time_now = start_time
delta_time = 0
progress = 0
rate = 0
eta = 0
total_units = len(slim_df.dialogue)
time_end = 0
for dialogue in slim_df.dialogue:
    time_now =  time.time()
    delta_time = time_now - start_time
    rate = delta_time / count
    remaining = total_units - count
    eta = remaining * rate
    time_end = (start_time + eta) #to time stamp string....

    dialogue = nltk.word_tokenize(dialogue)
    dialogue = [ word for word in dialogue if not word in set(sw)]
    lemma = nlp.WordNetLemmatizer()
    dialogue = [porter_stemmer.stem(word) for word in dialogue]
    dialogue = " ".join(dialogue)
    dialogue_list.append(dialogue)
    count = count + 1
    
dialogue_list

KeyboardInterrupt: ignored

# Split data into training and testing sets

In [ ]:
#Split data into training and testing 

#Tokenize the gender column into numbers



set